<a href="https://colab.research.google.com/github/jesus1080/CompuacionEvolutiva-/blob/master/12Vuelos3PistasCTrafico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CONTROL DE TRÁFICO AEREO
Asignacion de vuelos a pistas, teniendo en cuenta tiempos de retraso, tipos de avion, objetivo minimizar el tiempo para la asignaciond de los vuelos en las pistas disponibles.  

In [ ]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.2 MB/s eta 0:00:00


In [ ]:
import random
import array
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import numpy as np


In [ ]:
def initind(icls):
  individuo = np.array([
    np.random.choice(range(1, 13), size=12, replace=False),
    np.random.randint(1, 4, size=12)
  ])
  return icls(individuo)

In [ ]:
def init2d(icls, num_flights, num_runways):

    flights = np.random.randint(1, 11, size=(num_flights,))
    runways = np.random.randint(1, 11, size=(num_runways,))
    return icls([flights, runways])

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", np.ndarray, typecode="I", fitness=creator.FitnessMin, strategy=None)

In [ ]:
def separacionAux(pista,vuelo,vuelos):
      tv = ['w','g','w','w','p','w','g','w','w','p','w','g']
      dicsep = {'ww': 1, 'wg': 1.5, 'wp': 2, 'gw': 1, 'gg': 1.5, 'gp': 1.5, 'pw': 1, 'pg': 1, 'pp':1}
      pos = pista.index(vuelo)
      if pos == 0:
        return 0
      else:
        vueloant = pista[pos-1]
        posvuelo = np.where(vuelos == vuelo)
        posvueloant = np.where(vuelos == vueloant)
        t1 = tv[posvuelo[0][0]]
        t2 = tv[posvueloant[0][0]]
        return  dicsep.get(t1+t2)

In [ ]:
def separacion(p1,p2,p3,vuelo,vuelos):
  if vuelo in p1:
      return separacionAux(p1,vuelo,vuelos)
  elif vuelo in p2:
    return separacionAux(p2,vuelo,vuelos)
  elif vuelo in p3:
    return separacionAux(p3,vuelo,vuelos)

In [ ]:
def evalOneMax(ind):
    mtel = np.array([[11,15,6,6,9,7,15,6,6,9,7,9],[10,17,7,7,12,6,17,7,7,12,6,7],[9,19,8,8,15,5,19,8,8,15,5,5]])
    vuelos = ind[0, :12]
    pistas = ind[1, :12]
    p1 = []
    p2 = []
    p3 = []
    tels = []
    tlas = []
    tres = []
    for i in range(12):
      if(pistas[i]==2):
        p2.append(vuelos[i])
      elif pistas[i]==1:
        p1.append(vuelos[i])
      elif pistas[i]==3:
        p1.append(vuelos[i])
      tels.append(mtel[pistas[i]-1][vuelos[i]-1])

    for i in range(12):
      tlas.append(tels[i]+separacion(p1,p2,p1,vuelos[i],vuelos))

    for i in range(12):
      vuelop = vuelos[i] - 1
      minimo = np.min(mtel[:, vuelop])
      tres.append(tlas[i] - minimo)

    suma = sum(x**2 for x in tres)
    return suma,

In [ ]:
def cxTwoPointCopy(icls,ind1,ind2):

      #cabeza11 = ind1[0, :2]
      #cabeza12 = ind1[0, 2:]
      cola11 = ind1[1, :6]
      cola12 = ind1[1, 6:]

      #cabeza21 = ind2[0, :2]
      #cabeza22 = ind2[0, 2:]
      cola21 = ind2[1, :6]
      cola22 = ind2[1, 6:]

      arrCo = np.concatenate((cola11, cola21))
      arrayCo = np.concatenate((cola12, cola22))


      ind1 = icls(np.vstack((ind1[0],arrCo)))
      ind2 = icls(np.vstack((ind2[0],arrayCo)))

      return ind1, ind2




In [ ]:
def mutar(ind):

    index1 = random.randint(0, 11)
    index2 = random.randint(0, 11)

    # intercambio
    ind[0][index1], ind[0][index2] = ind[0][index2], ind[0][index1]
    ind[1][index1], ind[1][index2] = ind[1][index2], ind[1][index1]

    return ind,

In [ ]:
toolbox = base.Toolbox()
toolbox.register("individual", initind,creator.Individual )
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", cxTwoPointCopy,creator.Individual)
toolbox.register("mutate", mutar)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
def main():
    random.seed(64)

    pop = toolbox.population(n=300)

    # Numpy equality function (operators.eq) between two arrays returns the
    # equality element wise, which raises an exception in the if similar()
    # check of the hall of fame. Using a different equality function like
    # numpy.array_equal or numpy.allclose solve this issue.
    hof = tools.HallOfFame(1,similar=numpy.array_equal)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, stats=stats,
                      halloffame=hof)

    return pop, stats, hof

In [ ]:
if __name__ == "__main__":
    p,s,h=main()
    print(h[0])

gen	nevals	avg    	std    	min 	max  
0  	300   	118.035	36.2313	43.5	236.5
1  	169   	92.6458	28.615 	30.75	185.75
2  	148   	76.2925	24.704 	30.75	161.5 
3  	169   	66.7925	22.6028	28.75	154.25
4  	195   	62.195 	22.5005	23.5 	165.5 
5  	170   	59.7517	24.9403	19.5 	157   
6  	183   	54.3675	26.2551	19.5 	159.75
7  	195   	54.8342	29.2142	19.5 	168.5 
8  	178   	49.2117	27.2929	18.25	149.5 
9  	177   	45.3542	28.9181	16.5 	149.25
10 	192   	42.7583	27.057 	16.5 	152.75
11 	188   	46.0217	30.9895	16.5 	192.5 
12 	178   	43.6425	28.2987	16.5 	141.5 
13 	172   	46.3525	30.3959	16.5 	141.5 
14 	194   	45.9075	29.8879	16.5 	156.5 
15 	192   	48.6233	30.6898	16.5 	135.5 
16 	162   	41.1325	26.6674	16.5 	138.5 
17 	196   	45.3967	30.5922	16.5 	136.5 
18 	192   	46.5442	30.6672	16.5 	148.5 
19 	168   	41.9325	30.066 	16.5 	136.5 
20 	198   	45.86  	31.1553	16.5 	148.5 
[[ 8 10  7 12 11  3  2  5  9  4  6  1]
 [ 1  1  1  3  2  1  1  1  1  2  3  3]]
